In [1]:
from datetime import datetime #to get the current date
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import numpy as np

In [ ]:
def extract(job_title, location):
    inquiry = 'https://www.indeed.com/jobs?q={}&l={}&filter=0'
    job_title = job_title.replace(' ', '+')
    location = location.replace(' ', '+')
    url = inquiry.format(job_title, location)
    return url

In [ ]:
https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=&locT=N&locId=1&jobType=&context=Jobs&sc.keyword={}&dropdown=0